In [2]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.clustering import KMeans

from sklearn import preprocessing

import gensim
from gensim import corpora
import sys
import re
import json
import numpy as np
from sklearn.metrics import confusion_matrix
import pandas as pd

In [3]:
FILENAME_GL = "/home/dhuy237/thesis/code/bimetaReduce/bimeta/data/kmeans/output/OutStep_2_2"
FILENAME_CORPUS = "/home/dhuy237/thesis/code/bimetaReduce/bimeta/data/kmeans/output/OutStep_1_3"
FILENAME_LABELS = "/home/dhuy237/thesis/code/bimetaReduce/bimeta/data/kmeans/output/OutStep_1_1"
DICTIONARY_PATH = "/home/dhuy237/thesis/code/bimetaReduce/bimeta/data/kmeans/dictionary.pkl"

In [4]:
NUM_OF_SPECIES = 2
GROUP_AGGREGATION = "MEAN"  # MEAN or MEDIAN
SCALING = True
CLUSTERING_METHOD = "KMEANS"

In [5]:
def read_group(filename_gl):
    GL = []

    with open(filename_gl) as f:
        content_vertices = f.readlines()

    for line in content_vertices:
        clean_line = re.sub("[\t\n\[\]\'']", '', line).replace(' ', '').split(',')
        GL.append(list(map(int, clean_line))) # Convert all strings in a list to int

    return GL

def load_dictionary(dictionary_path):
    dictionary = corpora.Dictionary.load(dictionary_path)
    return dictionary

def read_corpus(filename_corpus):
    corpus = []

    with open(filename_corpus) as f:
        content_corpus = f.readlines()

    for line in content_corpus:
        clean_line = json.loads(line.replace('null\t', '{"a":').replace("\n", "}"))["a"][1]
        corpus.append(clean_line)
    
    return corpus

def read_labels(filename_labels):
    labels = []
    
    with open(filename_labels) as f:
        content_labels = f.readlines()
    
    for line in content_labels:
        clean_line = int(re.sub('[null\t\n\[\]\""]', '', line).replace(' ', '').split(',')[2])
        labels.append(clean_line)
    
    return labels

In [6]:
def compute_dist(dist, groups, seeds, only_seed=True):
    res = []
    if only_seed:
        for seednodes in seeds:
            tmp = dist[seednodes, :]
            if GROUP_AGGREGATION == "MEAN":
                res += [np.mean(tmp, axis=0)]
            elif GROUP_AGGREGATION == "MEDIAN":
                res += [np.median(tmp, axis=0)]
    else:
        for groupnodes in groups:
            tmp = dist[groupnodes, :]
            if GROUP_AGGREGATION == "MEAN":
                res += [np.mean(tmp, axis=0)]

            elif GROUP_AGGREGATION == "MEDIAN":
                res += [np.median(tmp, axis=0)]
                
    return np.array(res)

In [7]:
# Test read_group()
GL = []

with open(FILENAME_GL) as f:
    content_vertices = f.readlines()

for line in content_vertices:
    clean_line = re.sub("[\t\n\[\]\'']", '', line).replace(' ', '').split(',')
    GL.append(list(map(int, clean_line))) # Convert all strings in a list to int
GL[:5]

[[0, 362, 407], [1, 299], [2], [3], [4]]

In [8]:
labels = read_labels(FILENAME_LABELS)
print(len(labels))
labels[:5]

1281


[0, 0, 0, 0, 0]

In [9]:
def evalQuality(y_true, y_pred, n_clusters=NUM_OF_SPECIES):
    A = confusion_matrix(y_pred, y_true)
    if len(A) == 1:
      return 1, 1
    prec = sum([max(A[:,j]) for j in range(0,n_clusters)])/sum([sum(A[i,:]) for i in range(0,n_clusters)])
    rcal = sum([max(A[i,:]) for i in range(0,n_clusters)])/sum([sum(A[i,:]) for i in range(0,n_clusters)])

    return prec, rcal

In [10]:
def clustering(dictionary_path, filename_corpus, filename_gl, num_of_species):
    # Load dictionary, corpus and group list
    dictionary = load_dictionary(dictionary_path)
    corpus = read_corpus(filename_corpus)
    GL = read_group(filename_gl)

    corpus_m = gensim.matutils.corpus2dense(corpus, len(dictionary.keys())).T

    # Not implemented to get seed list yet
    SL = []

    # Training the clustering model
    kmer_group_dist = compute_dist(corpus_m, GL, SL, only_seed=False)

    model = cluster_groups(kmer_group_dist, num_of_species)
    
    y_kmer_grp_cl = assign_cluster_2_reads(GL, model)

    return y_kmer_grp_cl

def assign_cluster_2_reads( groups, y_grp_cl ):
    label_cl_dict=dict()

    for idx, g in enumerate(groups):
        for r in g:
            label_cl_dict[r]=y_grp_cl[idx]
    
    y_cl=[]
    for i in sorted (label_cl_dict):
        y_cl.append(label_cl_dict[i])

    return y_cl

In [11]:
dictionary = load_dictionary(DICTIONARY_PATH)
corpus = read_corpus(FILENAME_CORPUS)
GL = read_group(FILENAME_GL)

In [12]:
dictionary

In [13]:
corpus[:3]

[[[0, 21],
  [1, 4],
  [2, 5],
  [3, 18],
  [4, 4],
  [5, 4],
  [6, 1],
  [7, 10],
  [8, 2],
  [9, 2],
  [10, 3],
  [11, 1],
  [12, 18],
  [13, 2],
  [14, 4],
  [15, 14],
  [16, 3],
  [18, 1],
  [19, 6],
  [20, 5],
  [21, 3],
  [23, 5],
  [24, 2],
  [27, 1],
  [28, 3],
  [29, 3],
  [30, 3],
  [31, 1],
  [32, 1],
  [34, 2],
  [35, 3],
  [36, 1],
  [37, 1],
  [38, 5],
  [41, 1],
  [42, 4],
  [43, 2],
  [44, 3],
  [45, 19],
  [46, 4],
  [47, 4],
  [48, 5],
  [49, 3],
  [50, 4],
  [52, 3],
  [54, 2],
  [55, 6],
  [56, 5],
  [57, 3],
  [58, 8],
  [59, 2],
  [61, 4],
  [62, 3],
  [63, 1],
  [64, 1],
  [65, 3],
  [67, 5],
  [68, 6],
  [69, 1],
  [70, 5],
  [71, 5],
  [72, 2],
  [73, 3],
  [74, 5],
  [76, 1],
  [79, 2],
  [80, 4],
  [81, 1],
  [82, 1],
  [91, 7],
  [92, 3],
  [94, 3],
  [95, 4],
  [97, 4],
  [98, 3],
  [99, 14],
  [100, 2],
  [101, 1],
  [103, 2],
  [105, 2],
  [106, 1],
  [107, 1],
  [110, 1],
  [114, 2],
  [116, 1],
  [117, 1],
  [120, 1],
  [121, 2],
  [124, 2],
  [125, 3],

In [14]:
corpus_m = gensim.matutils.corpus2dense(corpus, len(dictionary.keys())).T

In [15]:
corpus_m

array([[21.,  4.,  5., ...,  2.,  1., 18.],
       [ 3.,  1.,  2., ...,  0.,  1.,  7.],
       [16.,  3., 14., ...,  7.,  4., 10.],
       ...,
       [ 5.,  3.,  4., ...,  4.,  9.,  6.],
       [23.,  7., 18., ..., 10.,  2.,  9.],
       [37.,  5.,  5., ..., 15.,  5., 11.]], dtype=float32)

In [16]:
GL[:5]

[[0, 362, 407], [1, 299], [2], [3], [4]]

In [17]:
len(GL)

1101

In [18]:
SL = []
kmer_group_dist = compute_dist(corpus_m, GL, SL, only_seed=False)

In [19]:
print(np.shape(kmer_group_dist))
kmer_group_dist

(1101, 136)


array([[20.666666 ,  2.6666667,  5.       , ...,  3.3333333,  2.       ,
        18.       ],
       [ 3.5      ,  1.       ,  2.       , ...,  0.       ,  1.       ,
         7.       ],
       [16.       ,  3.       , 14.       , ...,  7.       ,  4.       ,
        10.       ],
       ...,
       [17.       ,  6.       ,  6.       , ...,  2.       ,  3.       ,
        16.       ],
       [23.       ,  7.       , 18.       , ..., 10.       ,  2.       ,
         9.       ],
       [37.       ,  5.       ,  5.       , ..., 15.       ,  5.       ,
        11.       ]], dtype=float32)

In [20]:
df = pd.DataFrame(kmer_group_dist)

In [21]:
df.head()

,0,1,2,3,4,5,6,7,8,9,...,126,127,128,129,130,131,132,133,134,135
0,20.666666,2.666667,5.0,17.666666,4.666667,3.666667,1.0,8.333333,2.333333,2.666667,...,14.0,1.666667,1.666667,6.0,6.333333,8.666667,0.0,3.333333,2.0,18.0
1,3.500000,1.000000,2.0,10.000000,8.000000,7.500000,3.0,3.500000,1.000000,1.500000,...,8.0,0.000000,1.000000,4.5,8.500000,6.000000,1.5,0.000000,1.0,7.0
2,16.000000,3.000000,14.0,12.000000,2.000000,1.000000,0.0,3.000000,14.000000,2.000000,...,11.0,0.000000,9.000000,5.0,3.000000,4.000000,0.0,7.000000,4.0,10.0
3,4.000000,1.000000,7.0,12.000000,9.000000,1.000000,1.0,4.000000,4.000000,6.000000,...,9.0,7.000000,1.000000,9.0,1.000000,5.000000,0.0,6.000000,3.0,6.0
4,6.000000,3.000000,2.0,14.000000,2.000000,4.000000,1.0,5.000000,5.000000,3.000000,...,13.0,2.000000,3.000000,10.0,11.000000,5.000000,0.0,2.000000,1.0,18.0


In [22]:
group_dist_df = spark.createDataFrame(df)

In [23]:
group_dist_df.show()

+-----------------+------------------+----+-----------------+-----------------+------------------+---+-----------------+------------------+------------------+------------------+------------------+------------------+----+----+----+----+---+------------------+---+---+------------------+------------------+------------------+---+---+---+---+------------------+------------------+---+------------------+---+----+------------------+---+---+---+-----------------+------------------+---+------------------+---+------------------+------------------+-----------------+------------------+-----------------+-----------------+---+------------------+---+------------------+------------------+------------------+-----------------+-----------------+----+-----------------+------------------+---+---+---+---+---+------------------+---+-----------------+-----------------+------------------+-----------------+---+------------------+------------------+-----------------+---+------------------+------------------+---+-

In [24]:
df_columns = group_dist_df.schema.names

In [25]:
vecAssembler = VectorAssembler(inputCols=df_columns, outputCol="features")
new_df = vecAssembler.transform(group_dist_df)
new_df.show()

+-----------------+------------------+----+-----------------+-----------------+------------------+---+-----------------+------------------+------------------+------------------+------------------+------------------+----+----+----+----+---+------------------+---+---+------------------+------------------+------------------+---+---+---+---+------------------+------------------+---+------------------+---+----+------------------+---+---+---+-----------------+------------------+---+------------------+---+------------------+------------------+-----------------+------------------+-----------------+-----------------+---+------------------+---+------------------+------------------+------------------+-----------------+-----------------+----+-----------------+------------------+---+---+---+---+---+------------------+---+-----------------+-----------------+------------------+-----------------+---+------------------+------------------+-----------------+---+------------------+------------------+---+-

In [26]:
kmeans = KMeans(k=2, seed=1)  # 2 clusters here
model = kmeans.fit(new_df.select('features'))

In [27]:
transformed = model.transform(new_df)
transformed.select(["features", "prediction"]).show()  

+--------------------+----------+
|            features|prediction|
+--------------------+----------+
|[20.6666660308837...|         0|
|[3.5,1.0,2.0,10.0...|         0|
|(136,[0,1,2,3,4,5...|         1|
|[4.0,1.0,7.0,12.0...|         0|
|[6.0,3.0,2.0,14.0...|         0|
|[9.0,9.0,10.0,5.0...|         1|
|[14.0,7.0,5.0,14....|         0|
|[12.0,8.0,8.0,14....|         0|
|[6.0,8.0,4.0,16.0...|         0|
|[5.0,2.0,5.0,6.0,...|         0|
|[47.0,4.0,14.0,20...|         1|
|[42.5,7.0,15.5,17...|         1|
|[4.0,3.0,2.0,11.0...|         0|
|[22.0,7.0,9.0,22....|         1|
|[6.0,2.0,3.0,8.0,...|         0|
|(136,[0,1,2,3,4,7...|         1|
|[18.0,2.0,19.0,13...|         1|
|[11.0,2.0,10.0,12...|         1|
|[12.0,7.0,10.0,13...|         1|
|[38.0,7.0,13.0,17...|         1|
+--------------------+----------+
only showing top 20 rows



In [28]:
y_pred = transformed.select("prediction").rdd.flatMap(lambda x: x).collect()

In [29]:
y_pred[:5]

[0, 0, 1, 0, 0]

In [30]:
y_kmer_grp_cl = assign_cluster_2_reads(GL, y_pred)

In [31]:
prec, rcal = evalQuality(labels, y_kmer_grp_cl, n_clusters = NUM_OF_SPECIES)
print( 'K-mer (group): Prec = %.4f, Recall = %.4f, F1 = %.4f' % (prec, rcal, 2.0/(1.0/prec+1.0/rcal)) )

K-mer (group): Prec = 0.5480, Recall = 0.6097, F1 = 0.5772


In [35]:
def extract_read(filename_labels):
    labels = []
    
    with open(filename_labels) as f:
        content_labels = f.readlines()
    
    for line in content_labels:
        clean_line = str(re.sub('[null\t\n\[\]\""]', '', line).replace(' ', '').split(',')[1])
        labels.append(clean_line)
    
    return labels

In [36]:
reads = extract_read(FILENAME_LABELS)
reads[:5]

['AAACCCTCTTCCACGAACCCTCTTGAAAATCCCCCACATCCACAAAATAAATCAAATAAATTTCAACATTATCACCAAAAGGGTAAAAGGTTATTTAAAAAATAAAATAAATTTAAAAATTTAAATTAAATACCAAAAAAGCCAAATAACTTATTGTGATTCTTGAGCTTTCTTTAACTCTGCCTTCATATCTTGATAGACTTTAGTCCATTTTAATTTTCTTGGATTTCTTCCCATTCTGTAGCTTTTCTCACATTTGGATGAGCAGAAATATAATACAGTCCCATCTTTTTCTACGACCATTTTTCCTTTTCCTGGCTCAATTTCATAACCACAAAAGCTGCATGTTCTCCATTCTGGCATAGCTATCCCCCTTTAATAGTGTTTCAGTGATTTTAAAATAATTTAAGATTAAATTATTTATCTTCTTCTGTCTAATGGTCTTGCTTCTCTCTCTGTTTCTCTTAACATAATAATGTCTCCAACTTTAACTGGACCTTTAACGTTTCTAACTAAAACTCTTCCAGTATCTTTTCCACCTAAGATTTTACATCTAACTTGTATAATTCCTCCAGTAACCCCTGTTCTACCAATGACTTCAATAACTTCAGCAGCTACTGCTTCCTTATAAACAAATTCATCTTCCGATCCTCATCACCTAATATTAATGAAGGTTTAAAATTTATAAAAAAGTTAGTAGTAGTGTTTCATAATTTATATAATAATAACTATATACTATTGATTGATGGTTAAATAGCGTTCTAATAATTTACTGCTTCAAAACATTTACCTTTTCAATTAATACCTTTAACTCTTCAGCATCTCCTTCGTTG',
 'TAGCATGTAAATCCCTTATTTCTTAATTTCTCCCAGAATTATTTCTATTGCTTTATCAACTGCCTTGGCAACCTCTTCAGACAACCCTGGTTTTATGTCTGGCATTGTAAATTTTTACCTTGACAACCAATAACCACGACTTCTATGCCTTTATTATGTA

In [37]:
def save_file(result, output_path):
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    with open(output_path, 'w+') as f:
        for item in result:
            f.write("null\t%s\n" % json.dumps(item))

In [40]:
output_path_1 = "/home/dhuy237/thesis/code/bimetaReduce/bimeta/data/kmeans/output/group0.txt"
output_path_2 = "/home/dhuy237/thesis/code/bimetaReduce/bimeta/data/kmeans/output/group1.txt"

for i, read in enumerate(reads):
    if y_kmer_grp_cl[i] == 0:
        with open(output_path_1, 'a') as f1:
            f1.write("%s\t%s\n" % (i, read))
    elif y_kmer_grp_cl[i] == 1:
        with open(output_path_2, 'a') as f2:
            f2.write("%s\t%s\n" % (i, read))

In [42]:
def save_file(labels, reads, output_path):
    for i, read in enumerate(reads):
        with open(output_path+"/group_"+str(labels[i])+".txt", 'a') as f:
            f.write("%s\t%s\n" % (i, read))

In [45]:
output_path = "/home/dhuy237/thesis/code/bimetaReduce/bimeta/data/kmeans/output"
save_file(y_kmer_grp_cl, reads, output_path)